<a href="https://colab.research.google.com/github/laysabelici/pyspark/blob/main/9_criando_modelo_de_regressao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#INSTALAR O SPARK NO COLAB
!pip install pyspark py4j

#IMPORTAR SPARK NO COLAB
from pyspark.sql import SparkSession

#CRIAR SESSÃO SPARK
spark = SparkSession.builder.appName("ExemploPySpark").getOrCreate()

#OBTER CONTEXTO SPARK PARA A SESSÃO
sc = spark.sparkContext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=9cf5acf30af95e31ef2d5595b145a19169903418dd45cc043bfaab51e48ad2be
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
#IMPORTAR BIBLIOTECAS
from pyspark.ml.regression import LinearRegression, RandomForestRegressor

In [3]:
#IMPORTAR BIBLIOTECAS PARA AVALIAR PERFORMANCE DO MODELO
from pyspark.ml.evaluation import  RegressionEvaluator

In [4]:
#IMPORTAR BIBLIOTECAS HOTENDCODING
from pyspark.ml.feature import VectorAssembler

In [5]:
#INGESTÃO DE DADOS
from pyspark.sql.types import *

In [8]:
#IMPORT DADOS
carros_temp = spark.read.csv('/spark/Carros.csv', inferSchema=True, header= True, sep=';')
carros_temp.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [9]:
#VAMOS PREVER A POTÊNCIA DO CARRO

In [16]:
#SEPARAR AS COLUNAS DESEJADAS PARA O MODELO
carros = carros_temp.select('Consumo', 'Cilindros', 'Cilindradas', 'HP')
carros.show(5)

+-------+---------+-----------+---+
|Consumo|Cilindros|Cilindradas| HP|
+-------+---------+-----------+---+
|     21|        6|        160|110|
|     21|        6|        160|110|
|    228|        4|        108| 93|
|    214|        6|        258|110|
|    187|        8|        360|175|
+-------+---------+-----------+---+
only showing top 5 rows



In [12]:
#CRIAR O VETOR DE CARACTERISTICAS (VARIÁVEIS INDEPENDENTES)

In [17]:
#DEFINIR AS VARIÁVEIS INDEPENDENTES NO PARAMETRO
veccaracteristicas = VectorAssembler(inputCols=[('Consumo'), ('Cilindros'), ('Cilindradas')], outputCol='caracteristicas')

In [18]:
#AGORA VAMOS APLICAR O OBJETO CRIADO A VARIAVEL CARROS
carros = veccaracteristicas.transform(carros)
carros.show(5)

+-------+---------+-----------+---+-----------------+
|Consumo|Cilindros|Cilindradas| HP|  caracteristicas|
+-------+---------+-----------+---+-----------------+
|     21|        6|        160|110| [21.0,6.0,160.0]|
|     21|        6|        160|110| [21.0,6.0,160.0]|
|    228|        4|        108| 93|[228.0,4.0,108.0]|
|    214|        6|        258|110|[214.0,6.0,258.0]|
|    187|        8|        360|175|[187.0,8.0,360.0]|
+-------+---------+-----------+---+-----------------+
only showing top 5 rows



In [21]:
#DIVIDIR DADOS EM TREINO E TESTE
carrosTreino, carrosTeste = carros.randomSplit([0.7, 0.3])
print(carrosTreino.count())

22


In [22]:
print(carrosTeste.count())

10


In [23]:
#CRIANDO MODELO DE REGRESSÃO
reglin = LinearRegression(featuresCol='caracteristicas', labelCol='HP')
modelo = reglin.fit(carrosTreino)

In [24]:
#PREVISÃO USANDO MODELO CRIADO
previsao = modelo.transform(carrosTeste)
previsao.show(5)

+-------+---------+-----------+---+------------------+------------------+
|Consumo|Cilindros|Cilindradas| HP|   caracteristicas|        prediction|
+-------+---------+-----------+---+------------------+------------------+
|     21|        6|        160|110|  [21.0,6.0,160.0]|171.89689474260376|
|    133|        8|        350|245| [133.0,8.0,350.0]|215.74206164249188|
|    152|        8|       2758|180|[152.0,8.0,2758.0]|  188.985470439663|
|    155|        8|        318|150| [155.0,8.0,318.0]| 213.0743220660408|
|    173|        8|       2758|180|[173.0,8.0,2758.0]| 186.1323298717644|
+-------+---------+-----------+---+------------------+------------------+
only showing top 5 rows



In [25]:
#AVALIAR MODELO POR MÉTRICA DE PERFORMANCE
avaliar = RegressionEvaluator(predictionCol='prediction', labelCol='HP', metricName='rmse')
rmse = avaliar.evaluate(previsao)
print(rmse)

33.82672527105433


In [32]:
#GERAR NOVO MODELO COM DADOS DE TESTE
rfreg = RandomForestRegressor(featuresCol='caracteristicas', labelCol='HP')
modelo2 = rfreg.fit(carrosTreino)

In [33]:
previsao2 = modelo2.transform(carrosTeste)
previsao2.show(5)

+-------+---------+-----------+---+------------------+------------------+
|Consumo|Cilindros|Cilindradas| HP|   caracteristicas|        prediction|
+-------+---------+-----------+---+------------------+------------------+
|     21|        6|        160|110|  [21.0,6.0,160.0]|           125.475|
|    133|        8|        350|245| [133.0,8.0,350.0]| 227.7276262626263|
|    152|        8|       2758|180|[152.0,8.0,2758.0]|191.02554292929295|
|    155|        8|        318|150| [155.0,8.0,318.0]| 196.3410984848485|
|    173|        8|       2758|180|[173.0,8.0,2758.0]|167.79458333333332|
+-------+---------+-----------+---+------------------+------------------+
only showing top 5 rows



In [34]:
#GERAR UMA MÉTRICA PARA SEGUNDA PREVISÃO
rmse2 = avaliar.evaluate(previsao2)
print(rmse2)

28.595672235662512


In [35]:
print(rmse)

33.82672527105433
